In [1]:
pip install cplex

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install docplex

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
from docplex.cp.model import *

In [4]:
model=CpoModel()

In [5]:
import pandas as pd

In [6]:
classes_to_take=[]
val="class"
val=input("Enter the course you would like to take ")
while val.lower()!="exit":
    classes_to_take.append(val)
    val=input("Enter the course you would like to take ")
    

Enter the course you would like to take CS100
Enter the course you would like to take CS115
Enter the course you would like to take EXIT


In [29]:
#The real data dataframe
df2=pd.read_csv("CourseStatusReport--2022-04-02.csv")
df2 = df2.drop(columns=["Status", "Cross List Code", "College", "Department", "Schedule Type", "Max Enrollment", "Current Enrollment", "Enrollment Available", "Waitlist Count", "Campus", "Location", "Seating", "Dates"])
df2.head()

,CRN,Course Code,Course Subject,Course Number,Course Title,Credits,Days,Time,Primary Instructor,Secondary Instructors
0,10014,BIOL 100-01,BIOL,100,Intro to Profession,2,R,0840 - 1020,"Menhart, Nicholas",NaN
1,11189,BIOL 105-01,BIOL,105,Introduction to Biology,3,TR,1515 - 1630,"Irving, Thomas",NaN
2,10015,BIOL 107-01,BIOL,107,General Biol Lecture,3,TR,1125 - 1240,"Spink, Kathryn",NaN
3,10017,BIOL 109-L01,BIOL,109,General Biology Lab,1,W,1350 - 1630,"Lin, Yuting",NaN
4,14407,BIOL 109-L02,BIOL,109,General Biology Lab,1,W,1000 - 1240,"Lin, Yuting",NaN


In [30]:
#Testing: dataframe of actually every course at IIT fall 2022
df3 = pd.read_csv("CourseStatusReport--2022-04-16.csv")
df2 = df3.drop(columns=["Status", "Cross List Code", "College", "Department", "Schedule Type", "Max Enrollment", "Current Enrollment", "Enrollment Available", "Waitlist Count", "Campus", "Location", "Seating", "Dates"])
df2.head()

,CRN,Course Code,Course Subject,Course Number,Course Title,Credits,Days,Time,Primary Instructor,Secondary Instructors
0,11875,AAH 119-01,AAH,119,Hist of World Architecture I,3.0,MW,1125 - 1215,"Hazard, Erin",NaN
1,11876,AAH 119-D01,AAH,119,Discussion,0.0,M,0900 - 0950,NaN,NaN
2,11877,AAH 119-D02,AAH,119,Discussion,0.0,M,1000 - 1050,NaN,NaN
3,11878,AAH 119-D03,AAH,119,Discussion,0.0,M,1000 - 1050,NaN,NaN
4,11879,AAH 119-D04,AAH,119,Discussion,0.0,W,0900 - 0950,NaN,NaN


In [31]:
import random
random.seed(2022)
df2["Course Quality"]=[random.randrange(5) for i in range(len(df2))]
df2["Professor Difficulty"]=[random.randrange(5) for i in range(len(df2))]

In [32]:
df2["ID"] = range(len(df2))

In [33]:
model2=CpoModel()

In [34]:
#creating binary vars for each class
classVariables=[]
for index, row in df2.iterrows():
    classVariables.append(model2.integer_var(min=0, max=1, name="id"+str(row['ID'])))    
    #print("crn"+str(row['CRN']))
df2['Vars']=classVariables

In [35]:
#Course sections constraint
#Make a dictionary. Keys: Course subject course number touple. Values: variables
sections2 = {}
for idx, x in enumerate(df2["Course Code"]): #List through all Course Codes. Consider the place before the hyphen on "Course Code"
    #print(x.split("-")[0])
    x = x.split("-")[0]
    if(x not in sections2):
        sections2[x] = [classVariables[idx]]
    else:
        sections2[x].append(classVariables[idx])
#For each key in the dictionary, add a constraint
# for coursecode in sections2:
#     #For each constraint: the sum of all CRN's is equal to 1
#     allsectionssum = model2.sum(1*(sections2[coursecode][i]) for i in range(len(sections2[coursecode])))
#     model2.add_constraint(allsectionssum <= 1)
#     #print(allsectionssum)

In [36]:
#observe that constraints were added
model2.print_information()

Model: 1887679573
 - source file: C:/Users/nexen/AppData/Local/Temp/ipykernel_21124/1887679573.py
 - modeling time: 0.00 sec
 - number of integer variables:  0
 - number of interval variables: 0
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        0
 - number of root expressions:   0
 - number of expression nodes:   0
 - operations:                   None


In [37]:
#Course time slot constraints
#Make a dictionary. Keys: Course time slot. Values: ID's
timeslots = {}
for idx, x in enumerate(df2["Time"]): #List through all Time slots"
    if((type(df2["Days"][idx]))!=float):
        for char in (df2["Days"][idx]):
            timeday = char+x
            if(timeday not in timeslots):
                timeslots[timeday] = [classVariables[idx]]
            else:
                timeslots[timeday].append(classVariables[idx])
#For each key in the dictionary, add a constraint
for timeslot in timeslots:
    #For each constraint: the sum of all CRN's is equal to 1
    alltimeslotssum = model2.sum(1*(timeslots[timeslot][i]) for i in range(len(timeslots[timeslot])))
    model2.add_constraint(alltimeslotssum <= 1)


In [38]:
timeslots

{'M1125 - 1215': [<docplex.cp.expression.CpoIntVar at 0x24d8c9f5d00>,
 'W1125 - 1215': [<docplex.cp.expression.CpoIntVar at 0x24d8c9f5d00>,
 'M0900 - 0950': [<docplex.cp.expression.CpoIntVar at 0x24d8c468680>,
 'M1000 - 1050': [<docplex.cp.expression.CpoIntVar at 0x24d8c2bd380>,
 'W0900 - 0950': [<docplex.cp.expression.CpoIntVar at 0x24d8e612580>,
 'W1000 - 1050': [<docplex.cp.expression.CpoIntVar at 0x24d8c467a80>,
 'T1000 - 1115': [<docplex.cp.expression.CpoIntVar at 0x24d8c467900>,
 'R1000 - 1115': [<docplex.cp.expression.CpoIntVar at 0x24d8c467900>,
 'T1125 - 1240': [<docplex.cp.expression.CpoIntVar at 0x24d8c467f00>,
 'R1125 - 1240': [<docplex.cp.expression.CpoIntVar at 0x24d8c467f00>,
 'M1350 - 1505': [<docplex.cp.expression.CpoIntVar at 0x24d8c467500>,
 'W1350 - 1505': [<docplex.cp.expression.CpoIntVar at 0x24d8c467500>,
 'F1125 - 1305': [<docplex.cp.expression.CpoIntVar at 0x24d8c467ec0>],
 'T1000 - 1240': [<docplex.cp.expression.CpoIntVar at 0x24d8c467a40>,
 'T1350 - 1630': [<

In [39]:
#observe that constraints were added
model2.print_information()

Model: 1887679573
 - source file: C:/Users/nexen/AppData/Local/Temp/ipykernel_21124/1887679573.py
 - modeling time: 0.58 sec
 - number of integer variables:  1083
 - number of interval variables: 0
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        224
 - number of root expressions:   224
 - number of expression nodes:   5205
 - operations:                   lessOrEqual: 224, sum: 224, times: 1613


In [40]:
# #Credit count: The sum of the credit count for all taken sections is less that 18 and bigger than 12
# creditCount = [0]*len(df2)
# for i in range(len(df2)):
#     creditCount[i]=df2["Credits"][i]
# creditCount[:5]

In [41]:
allcreditcountsum = model2.sum(df2["Credits"][i] * df2["Vars"][i] for i in range(len(df2)))
model2.add_constraint(allcreditcountsum <= 18)
model2.add_constraint(allcreditcountsum >= 12)

In [42]:
#observe that constraints were added
model2.print_information()

Model: 1887679573
 - source file: C:/Users/nexen/AppData/Local/Temp/ipykernel_21124/1887679573.py
 - modeling time: 3.22 sec
 - number of integer variables:  2619
 - number of interval variables: 0
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        226
 - number of root expressions:   226
 - number of expression nodes:   11985
 - operations:                   greaterOrEqual: 1, lessOrEqual: 225, sum: 225, times: 4232


In [43]:
required_classes=[]
val="class"
val=input("Enter a course you need to take ")
while val.lower()!="exit":
    required_classes.append(val)
    val=input("Enter another course you need to take. \nType \"Exit\" to stop adding courses ")

Enter a course you need to take CHE 351
Enter another course you need to take. 
Type "Exit" to stop adding courses CHE 302
Enter another course you need to take. 
Type "Exit" to stop adding courses exit


In [44]:
required_classes

['CHE 351', 'CHE 302']

In [46]:
requested_classes=[]
val="class"
val=input("Enter a course you want to take ")
while val.lower()!="exit":
    requested_classes.append(val)
    val=input("Enter another course you want to take. \nType \"Exit\" to stop adding courses ")

Enter a course you want to take PSYC 301
Enter another course you want to take. 
Type "Exit" to stop adding courses PSYC 303
Enter another course you want to take. 
Type "Exit" to stop adding courses PSYC 221
Enter another course you want to take. 
Type "Exit" to stop adding courses exit


In [47]:
requested_classes

['PSYC 301', 'PSYC 303', 'PSYC 221']

In [48]:
#Seeing if coursecode keys are strings
for coursecode in sections2:
    print(coursecode, " ", type(coursecode))

AAH 119   <class 'str'>
AAH 322   <class 'str'>
AAH 380   <class 'str'>
ARCH 100   <class 'str'>
ARCH 107   <class 'str'>
ARCH 113   <class 'str'>
ARCH 201   <class 'str'>
ARCH 207   <class 'str'>
ARCH 305   <class 'str'>
ARCH 331   <class 'str'>
ARCH 334   <class 'str'>
ARCH 403   <class 'str'>
ARCH 413   <class 'str'>
ARCH 417   <class 'str'>
ARCH 419   <class 'str'>
ARCH 431   <class 'str'>
ARCH 433   <class 'str'>
ARCH 437   <class 'str'>
ARCH 438   <class 'str'>
ARCH 445   <class 'str'>
ARCH 449   <class 'str'>
ARCH 456   <class 'str'>
ARCH 471   <class 'str'>
ARCH 473   <class 'str'>
ARCH 482   <class 'str'>
ARCH 485   <class 'str'>
ARCH 488   <class 'str'>
ARCH 490   <class 'str'>
ARCH 491   <class 'str'>
ARCH 497   <class 'str'>
ARCH 500   <class 'str'>
ARCH 502   <class 'str'>
ARCH 506   <class 'str'>
ARCH 508   <class 'str'>
ARCH 509   <class 'str'>
ARCH 513   <class 'str'>
ARCH 520   <class 'str'>
ARCH 541   <class 'str'>
ARCH 543   <class 'str'>
ARCH 545   <class 'str'>
ARC

In [49]:
#Adding constraints for what courses to take, and identical course conflict avoidance
for coursecode in sections2:
    #For each constraint: the sum of all CRN's is equal to 1
    allsectionssum = model2.sum(1*(sections2[coursecode][i]) for i in range(len(sections2[coursecode])))
    if(coursecode in required_classes):
        model2.add_constraint(allsectionssum == 1)
    elif(coursecode in requested_classes):
        model2.add_constraint(allsectionssum <= 1)
    else:
        model2.add_constraint(allsectionssum == 0)

In [50]:
#creating minimization function (course quality and prof difficulty)
min_func=0
for index, row in df2.iterrows():
    min_func+=((5-row['Course Quality'])+row['Professor Difficulty'])*row['Vars']
min_func=model2.minimize(min_func)
model2.add(min_func)
min_func

In [51]:
#observe that constraints were added
model2.print_information()

Model: 1887679573
 - source file: C:/Users/nexen/AppData/Local/Temp/ipykernel_21124/1887679573.py
 - modeling time: 41.49 sec
 - number of integer variables:  2619
 - number of interval variables: 0
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        1115
 - number of root expressions:   1115
 - number of expression nodes:   28634
 - operations:                   equal: 885, greaterOrEqual: 1, lessOrEqual: 228, minimize: 1, plus: 2619, sum: 1113, times: 9470


In [52]:
#solving model
sol2=model2.solve()
sol2.print_solution()

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 2619 variables, 1114 constraints
 ! Presolve      : 3 extractables eliminated
 ! Initial process time : 0.03s (0.03s extraction + 0.00s propagation)
 !  . Log search space  : 5.0 (before), 5.0 (after)
 !  . Memory usage      : 3.9 MB (before), 3.9 MB (after)
 ! Using parallel search with 16 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          5                 -
 + New bound is 9
                        0          5    1            -
 + New bound is 15
                        0          5    1   F        -
 + New bound is 16
 *            24        6  0.09s        1      (gap is 33.33%)
 *            23       11  0.09s        1      (gap is 30.43%)
              23       46          1    1            -
 + New bound is 23 (gap is 0.00%)
 ! ----------

In [53]:
for index, row in df2.iterrows():
    #print(sol2.get_value("id"+str(index)))
    if(sol2.get_value("id"+str(index))!=0):
        print(df2["Course Code"][index])

CHE 302-02
CHE 351-01
PSYC 301-01
PSYC 303-01


In [191]:
df2.head()

,CRN,Course Code,Course Subject,Course Number,Course Title,Credits,Days,Time,Primary Instructor,Secondary Instructors,Course Quality,Professor Difficulty,ID,Vars
0,11875,AAH 119-01,AAH,119,Hist of World Architecture I,3.0,MW,1125 - 1215,"Hazard, Erin",NaN,4,2,0,"id0 = intVar(0, 1)"
1,11876,AAH 119-D01,AAH,119,Discussion,0.0,M,0900 - 0950,NaN,NaN,2,4,1,"id1 = intVar(0, 1)"
2,11877,AAH 119-D02,AAH,119,Discussion,0.0,M,1000 - 1050,NaN,NaN,3,0,2,"id2 = intVar(0, 1)"
3,11878,AAH 119-D03,AAH,119,Discussion,0.0,M,1000 - 1050,NaN,NaN,4,2,3,"id3 = intVar(0, 1)"
4,11879,AAH 119-D04,AAH,119,Discussion,0.0,W,0900 - 0950,NaN,NaN,2,0,4,"id4 = intVar(0, 1)"


In [42]:
#constraint: for classes in classes_to_take, the value must be 1
df=pd.read_csv("Sample-Data.csv")
df.head()

,Course Name,CRN,Credits,Days,Time,Course Quality,Professor Difficulty
0,CS100,1,3,F,1350 - 1505,1,1
1,CS100,2,3,M,1515 - 1630,2,1
2,CS100,3,3,MW,1700 - 1815,3,3
3,CS115,4,3,TR,1825 - 1940,1,2
4,CS115,5,3,F,1350 - 1505,2,2


In [43]:
#creating binary vars for each class
classes=[]
for index, row in df.iterrows():
    classes.append(model.integer_var(min=0, max=1, name="crn"+str(row['CRN'])))    
    #print("crn"+str(row['CRN']))
df['Vars']=classes

In [44]:
df.head()

,Course Name,CRN,Credits,Days,Time,Course Quality,Professor Difficulty,Vars
0,CS100,1,3,F,1350 - 1505,1,1,"crn1 = intVar(0, 1)"
1,CS100,2,3,M,1515 - 1630,2,1,"crn2 = intVar(0, 1)"
2,CS100,3,3,MW,1700 - 1815,3,3,"crn3 = intVar(0, 1)"
3,CS115,4,3,TR,1825 - 1940,1,2,"crn4 = intVar(0, 1)"
4,CS115,5,3,F,1350 - 1505,2,2,"crn5 = intVar(0, 1)"


In [45]:
for x in classes_to_take:
    #add constraint that at most ONE of the classes with the course name can be taken
    model.add_constraint(df.loc[df['Course Name'] == x]['Vars'].sum()<=1)
#don't have to do credit constraint because of the above constraint

In [11]:
#observe that constraints were added
model.print_information()

Model: 3524116639
 - source file: C:/Users/nexen/AppData/Local/Temp/ipykernel_21492/3524116639.py
 - modeling time: 19.48 sec
 - number of integer variables:  5
 - number of interval variables: 0
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        2
 - number of root expressions:   2
 - number of expression nodes:   12
 - operations:                   equal: 2, plus: 3


In [12]:
#creating minimization function (course quality and prof difficulty)
min_func=0
for index, row in df.iterrows():
    min_func+=((5-row['Course Quality'])+row['Professor Difficulty'])*row['Vars']

min_func

In [13]:
#solving model
min_func=model.minimize(min_func)
model.add(min_func)
sol=model.solve()
sol.print_solution()

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 8 variables, 2 constraints
 ! Presolve      : 4 extractables eliminated
 ! Initial process time : 0.02s (0.02s extraction + 0.01s propagation)
 !  . Log search space  : 8.0 (before), 8.0 (after)
 !  . Memory usage      : 267.6 kB (before), 267.6 kB (after)
 ! Using parallel search with 16 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          8                 -
 + New bound is 0
                        0          8    1            -
 + New bound is 5
                        0          8    1   F        -
 + New bound is 6
 *            10        6  0.05s        1      (gap is 40.00%)
 *             9       37  0.05s        1      (gap is 33.33%)
               9       60          1    1            -
 + New bound is 9 (gap is 0.00%)
 ! ---------------